In [1]:
# ---- Programming backprop from scratch -----
##########################
#   Importing libraries  #
##########################
import numpy as np
import pandas as pd

/home/joshuah/venv3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
###########################
#   Activation Functions  #
###########################
ReLU = np.vectorize(lambda z: np.fmax(0,z))
sigmoid = lambda z: 1 / (1 + np.exp(-z))
softmax = lambda z: np.exp(z)/(np.sum(np.exp(z),axis=1))[:,np.newaxis]
tanh = lambda z: np.tanh(z)

In [3]:
#######################
#  Utility Functions  #
#######################
def predict(Y_hat):
    return np.argmax(Y_hat, axis=1)

def error_rate(Y_hat, cl):
    prediction = predict(Y_hat)
    return np.mean(prediction != cl)

def cost(Y_hat, Y):
    tot = Y * np.log(Y_hat)
    return -tot.sum()

In [4]:
####################################
#   1- Hidden Layer ReLU Network   #
####################################
def forward(X,parameters):
    # Unpacking parameters    
    W,b1,V,b2 = parameters
    
    # Forward pass
    a1 = X.dot(W) + b1
    H = sigmoid(a1)
    a2 = H.dot(V) + b2
    Y_hat = softmax(a2)
    return H,Y_hat

In [ ]:
####################################
#   2 - Hidden Layer ReLU Network   #
####################################
def forward(X,parameters):
    # Unpacking parameters    
    W1,b1,W2,b2,V,v3 = parameters
    
    # Forward pass
    a1 = X.dot(W1) + b1
    H1 = sigmoid(a1)
    a2 = H.dot(W2) + b2
    H2 = sigmoid(a2)
    a3 = H2.dot(V) + b3
    Y_hat = softmax(a3)
    return H1,H2,Y_hat

In [25]:
################
#   Gradient   #
################

def grad(X,H,Y,Y_hat,parameters):  
    # Unpacking parameters    
    W,b1,V,b2 = parameters
  
    # Gradients - sigmoid
    dW = X.T.dot((Y_hat-Y).dot(V) * (H * (1 - H)))
    db1 = (Y_hat-Y).dot(V) * (H * (1 - H)).sum(axis=0)
    dV = H.T.dot(Y_hat - Y)
    db2 = (Y_hat - Y).sum(axis=0)
    
    #return {'dV':dV,'db2':db2,'dW':dW,'db1':db1}
    return dW,db1,dV,db2


In [26]:
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#   Parameter Update: Momentum + Regularization   #
###################################################
def parameter_update(parameters,  grads, 
                     momentum_params = [0,0,0,0], 
                     lr = 1, reg = 0, alpha = 0):
    # Unpacking parameters            
    W,b1,V,b2 = parameters
    dW,db1,dV,db2 = grads
    vW,vb1,vV,vb2 = momentum_params
    
    # Momentum update
    vW  = alpha * vW -  lr * (dW + reg*W)
    vb1 = alpha * vb1 - lr * (db1 + reg*b1)
    vV  = alpha * vV -  lr * (dV + reg*V)
    vb2 = alpha * vb2 - lr * (db2 + reg*b2)
    momentum_params = [vW,vb1,vV,vb2] 
    
    # Parameter updates
    W  = W  + vW
    b1 = b1 + vb1
    V  = V  + vV
    b2 = b2 + vb2
    parameters =[W,b1,V,b2]

    return parameters, momentum_params

In [27]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#      Building the model     #
###############################
def run_model(X, Y, cl,
              iterations = 1000,
              regularization_include = False,
              momentum_include = False):    

    ###################################
    #   Initial values for network    #
    ###################################
    # Intialize weights
    np.random.seed(123)
    W = np.random.randn(6).reshape(2,3)
    b1 = 0
    V = np.random.randn(9).reshape(3,3)
    b2 = 0
    parameters = [W,b1,V,b2]
    
    # Hyperparameters 
    lr = 0.0001 # learning rate
    reg = 0.01 * regularization_include
    
    # Momentum parameters
    alpha = 0.9 * momentum_include
    vV = 0
    vb2 = 0
    vW = 0
    vb1 = 0
    momentum_params = [vW,vb1,vV,vb2]    
    
    # Place holder for losses
    losses = []
    errors = []    
   
    ###################
    #   Run the model #
    ###################
    for i in range(0,iterations):
        # -- Forward propoagation --
        H,Y_hat = forward(X,parameters)
        
        # -- Backward propagation --
        # Gradient calculation
        grads_in = grad(X,H,Y,Y_hat,parameters)
        # Parameter update
        new_params, new_mom_param = parameter_update(parameters, grads_in, 
                             momentum_params, alpha = alpha, 
                             lr = lr, reg = reg)
        
        # -- Updating values --
        H,Y_hat = forward(X,new_params)
        parameters = new_params
        momentum_params = new_mom_param
        
        # Prediction and Error rate            
        errs_i = error_rate(Y_hat, cl) ; errors.append(errs_i)
        loss_i = cost(Y_hat, Y); losses.append(loss_i)
        if ((i % 25) == 0):    
            print(
            '''
            ---- Iteration {i} ----
            Error rate : {er}
            Loss: {loss}
            '''.format(i= i, er = errs_i, loss = loss_i))
    return {"errors":errs_i, "loss_i":loss_i, "parameters":parameters}

In [28]:
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#   Generate some training    #
#      data from a GMM        #
###############################
def gen_gmm_data(n = 999, plot=False):
    # Fixing seed for repeatability
    np.random.seed(123)
    
    # Parameters of a normal distribuion
    mean_1 = [0, 2] ; mean_2 = [2, -2] ; mean_3 = [-2, -2]
    mean = [mean_1, mean_2, mean_3] ; cov = [[1, 0], [0, 1]]  
    
    # Setting up the class probabilities
    n_samples = n
    pr_class_1 = pr_class_2 = pr_class_3 = 1/3.0
    n_class = (n_samples * np.array([pr_class_1,pr_class_2, pr_class_3])).astype(int)
  
    # Generate sample data
    for i in range(3):
        x1,x2 = np.random.multivariate_normal(mean[i], cov, n_class[i]).T
        if (i==0):
            xs = np.array([x1,x2])
            cl = np.array([n_class[i]*[i]])
        else: 
            xs_new = np.array([x1,x2])
            cl_new = np.array([n_class[i]*[i]])
            xs = np.concatenate((xs, xs_new), axis = 1)
            cl = np.concatenate((cl, cl_new), axis = 1)
    
    # Plot?
    if plot:
        matplotlib.pyplot.scatter(xs[:1,:],xs[1:,:], c = cl)

    # One hot encoding classes
    y = pd.Series(cl[0].tolist())
    y = pd.get_dummies(y).as_matrix() 

    # Normalizing data (prevents overflow errors)     
    mu = xs.mean(axis = 1)
    std = xs.std(axis = 1)
    xs = (xs.T - mu) / std
    
    return xs, y, cl
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#  Generate data for network    #
#################################
X, Y, cl = gen_gmm_data()

In [29]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#   Running Different Models  #
###############################
vanilla_sgd = run_model(X,Y,cl, iterations = 100,
              regularization_include = False,
              momentum_include = False)


            ---- Iteration 0 ----
            Error rate : 0.36536536536536535
            Loss: 815.5770638412325
            

            ---- Iteration 25 ----
            Error rate : 0.15315315315315314
            Loss: 567.097113716236
            

            ---- Iteration 50 ----
            Error rate : 0.01901901901901902
            Loss: 469.01670271879595
            

            ---- Iteration 75 ----
            Error rate : 0.005005005005005005
            Loss: 387.83445409994266
            
